# SMOTE all minority classes to (number of data set samples / classes)

In [1]:
from collections import Counter
import os
import pandas as pd
import numpy as np
import tensorflow as tf

data = np.load('/home/hll/IDS/2020/data/select/zuizhong/data_train.npy')
label= np.load('/home/hll/IDS/2020/data/select/zuizhong/label6_train.npy')

In [2]:
X=np.array(data)  
b=np.array(label)
bb=b.reshape(b.shape[0],)     
y10 = np.int32(bb) 

In [3]:
X.shape

(3167779, 12)

In [4]:
sorted(Counter(y10).items())

[(0, 2443895), (1, 200334), (2, 359), (3, 458010), (4, 65144), (5, 37)]

In [5]:
from imblearn.over_sampling import SMOTE
import time
time_start = time.time()
a = 527963

smo = SMOTE(ratio={1:a,2:a,3:a,4:a,5:a},random_state=42)  

X_smo, y_smo = smo.fit_sample(X, y10)   
print(sorted(Counter(y_smo).items()))

time_end = time.time()
time = time_end - time_start
print("time:",time)

Using TensorFlow backend.


[(0, 2443895), (1, 527963), (2, 527963), (3, 527963), (4, 527963), (5, 527963)]
time: 1978.3734097480774


In [6]:
X_smo.shape[0]

5083710

# Extract Majority class of data

In [7]:
list0 = []  
list1 = []  
list2 = []  

for i in range(X_smo.shape[0]):
    if y_smo[i] == 0:
        list0.append(X_smo[i])
    else:
        list1.append(X_smo[i])
        list2.append(y_smo[i])
    


In [8]:
data0 = np.array(list0)  
data1 = np.array(list1)
label1 = np.array(list2)

label11 = label1.reshape(label1.shape[0],)    

print("Normal class data shape：",data0.shape)
print("Attack class data shape：",data1.shape) 
print("Attack class label shape：",label11.shape)

Normal class data shape： (2443895, 12)
Attack class data shape： (2639815, 12)
Attack class label shape： (2639815,)


In [9]:
label11

array([3, 1, 3, ..., 5, 5, 5], dtype=int32)

# Cluster majority data into  C (total number of classes)

In [10]:
from sklearn.mixture import GaussianMixture
import time
time_start = time.time()

estimator = GaussianMixture(n_components=6)
estimator.fit(data0) 

time_end = time.time()
time = time_end - time_start
print("time:",time)

label_pred = estimator.predict(data0) 

time: 173.31280207633972


In [11]:
sorted(Counter(label_pred).items())

[(0, 68087), (1, 762293), (2, 701472), (3, 110673), (4, 176277), (5, 625093)]

# Select a certain amount of data from each cluster to form a new majority data

In [12]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []



s0=s1=s2=s3=s4=s5=0

for i in range(data0.shape[0]):
    if label_pred[i] == 0:
        c0.append(data0[i])
        s0=s0+1
    elif label_pred[i] == 1:
        c1.append(data0[i])
        s1=s1+1
    elif label_pred[i] == 2:
        c2.append(data0[i])
        s2=s2+1
    elif label_pred[i] == 3:
        c3.append(data0[i])
        s3=s3+1
    elif label_pred[i] == 4:
        c4.append(data0[i])
        s4=s4+1
    elif label_pred[i] == 5:
        c5.append(data0[i])
        s5=s5+1

        


In [13]:
a = 91975

In [15]:
# del c1[a:len(c1)]
del c2[a:len(c2)]
del c3[a:len(c3)]
del c4[a:len(c4)]
del c5[a:len(c5)]


In [16]:
c00 = np.array(c0)
c11 = np.array(c1)
c22 = np.array(c2)
c33 = np.array(c3)
c44 = np.array(c4)
c55 = np.array(c5)


In [17]:
q = np.concatenate((c00,c11,c22,c33,c44,c55),axis=0)
q.shape

(527962, 12)

In [18]:
label_zc = np.zeros((q.shape[0],), dtype=int)
label_zc.shape

(527962,)

In [19]:
data_end = np.concatenate((q,data1),axis=0)
label_end = np.concatenate((label_zc,label1),axis=0)

In [20]:
sorted(Counter(label_end).items())

[(0, 527962), (1, 527963), (2, 527963), (3, 527963), (4, 527963), (5, 527963)]

In [21]:
label_end = label_end.reshape(label_end.shape[0],1)

In [22]:
np.save("/home/hll/IDS/2020/data/select/zuizhong/SGM_data_train.npy",data_end)
np.save("/home/hll/IDS/2020/data/select/zuizhong/SGM_label6_train.npy",label_end)